# Lab 6 
## SSC442
### Isaac Updike 
### 10/10/2020

In [32]:
library(dslabs)
library(ggplot2)
library(dplyr)

Starter Code for grabbing the data:

In [33]:
ameslist <- read.table("https://msudataanalytics.github.io/SSC442/Labs/data/ames.csv",
                 header = TRUE,
                 sep = ",")


I like to take a quick peak

In [34]:
colnames(ameslist)
head(ameslist,3)

[1] "Id"            "MSSubClass"    "MSZoning"      "LotFrontage"  
 [5] "LotArea"       "Street"        "Alley"         "LotShape"     
 [9] "LandContour"   "Utilities"     "LotConfig"     "LandSlope"    
[13] "Neighborhood"  "Condition1"    "Condition2"    "BldgType"     
[17] "HouseStyle"    "OverallQual"   "OverallCond"   "YearBuilt"    
[21] "YearRemodAdd"  "RoofStyle"     "RoofMatl"      "Exterior1st"  
[25] "Exterior2nd"   "MasVnrType"    "MasVnrArea"    "ExterQual"    
[29] "ExterCond"     "Foundation"    "BsmtQual"      "BsmtCond"     
[33] "BsmtExposure"  "BsmtFinType1"  "BsmtFinSF1"    "BsmtFinType2" 
[37] "BsmtFinSF2"    "BsmtUnfSF"     "TotalBsmtSF"   "Heating"      
[41] "HeatingQC"     "CentralAir"    "Electrical"    "X1stFlrSF"    
[45] "X2ndFlrSF"     "LowQualFinSF"  "GrLivArea"     "BsmtFullBath" 
[49] "BsmtHalfBath"  "FullBath"      "HalfBath"      "BedroomAbvGr" 
[53] "KitchenAbvGr"  "KitchenQual"   "TotRmsAbvGrd"  "Functional"   
[57] "Fireplaces"    "FireplaceQu"   "GarageType"    "GarageYrBlt"  
[61] "GarageFinish"  "GarageCars"    "GarageArea"    "GarageQual"   
[65] "GarageCond"    "PavedDrive"    "WoodDeckSF"    "OpenPorchSF"  
[69] "EnclosedPorch" "X3SsnPorch"    "ScreenPorch"   "PoolArea"     
[73] "PoolQC"        "Fence"         "MiscFeature"   "MiscVal"      
[77] "MoSold"        "YrSold"        "SaleType"      "SaleCondition"
[81] "SalePrice"

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500


Starter code making a linear regression model relating GrLivArea to SalePrice, then putting the model directly into the plot function to get some data on model fit 

In [35]:
lm.fit = lm(SalePrice ~ GrLivArea, data=ameslist)

#plot(lm.fit)  #uncomment to show plotted residuals,Q-Q, etc

#plot(SalePrice ~ GrLivArea, data = ameslist, cex = 2) # plot data points
#abline(lm.fit, lwd = 3, col = "red") # add regression line

## Part 1: The value of a garage

Lets start by creating a dummy variable denoting whether there is a garage, which we determine to be false if garagecars = 0.  
  
We then create a simple linear model where the coeficient on cars describes the value difference estimated between a house with and without a garage. We estimate garages are estimated to be worth 82162 on average.

In [36]:
garage <- ameslist %>%
    
    mutate(hasgarage = if_else(GarageCars==0,0,1)) %>%
    select(SalePrice,hasgarage)

garagemod <- lm(SalePrice ~ hasgarage,data=garage)
summary(garagemod)



Call:
lm(formula = SalePrice ~ hasgarage, data = garage)

Residuals:
    Min      1Q  Median      3Q     Max 
-150169  -50480  -16398   29920  569520 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   103317       8579  12.043   <2e-16 ***
hasgarage      82162       8827   9.308   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 77210 on 1458 degrees of freedom
Multiple R-squared:  0.05609,	Adjusted R-squared:  0.05544 
F-statistic: 86.64 on 1 and 1458 DF,  p-value: < 2.2e-16


## 2. Multiple regression

Lets start by just selecting the numeric columns to regress. If we wanted to include categorical variables, maybe we could use dummies, but that'd be a lotta dummies. The . tells lm to use all other variables in df other than SalePrice. 

In [37]:
#cols <- colnames(ameslist)
#cols <- cols[!cols == "SalePrice"]
df <- select_if(ameslist, is.numeric)
mod <- lm(SalePrice ~ ., data = df)
summary(mod)
#summary(mod)[["coefficients"]][,"t value"] # inspect just the t-values


Call:
lm(formula = SalePrice ~ ., data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-442182  -16955   -2824   15125  318183 

Coefficients: (2 not defined because of singularities)
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -3.351e+05  1.701e+06  -0.197 0.843909    
Id            -1.205e+00  2.658e+00  -0.453 0.650332    
MSSubClass    -2.001e+02  3.451e+01  -5.797 8.84e-09 ***
LotFrontage   -1.160e+02  6.126e+01  -1.894 0.058503 .  
LotArea        5.422e-01  1.575e-01   3.442 0.000599 ***
OverallQual    1.866e+04  1.482e+03  12.592  < 2e-16 ***
OverallCond    5.239e+03  1.368e+03   3.830 0.000135 ***
YearBuilt      3.164e+02  8.766e+01   3.610 0.000321 ***
YearRemodAdd   1.194e+02  8.668e+01   1.378 0.168607    
MasVnrArea     3.141e+01  7.022e+00   4.473 8.54e-06 ***
BsmtFinSF1     1.736e+01  5.838e+00   2.973 0.003014 ** 
BsmtFinSF2     8.342e+00  8.766e+00   0.952 0.341532    
BsmtUnfSF      5.005e+00  5.277e+00   0.948 0.343173    
TotalB

Several predictors appear strongly related to the responce. the * , ** , and *** tags in the summary denote significance at the 5%, 1% and 0.1% levels. Some of the predictors with the highest statistical significance are MSSubClass, LotArea, OverallQual, OverallCond, YearBuilt, MasVnrArea, X1stFlrSF, X2ndFlrSF, BedroomAbvGr, TotRmsAbvGrd, and GarageCars.  
  
There are several variables with 'year' in the name but only YearBuilt is signicant. The coeficient of YearBuilt is 316.4, meaning for a 1 unit increase in year (or a house 1 year newer), there is an expected $316.4 increase in sale price.  
  
Two variables have been omitted from the model, supposedly because of "singularities". Didn't know R was into the matrix. We can incpect co-linearity with allias() to see if variables are perfectly predictive of each other.

In [38]:
alias(mod)

Model :
SalePrice ~ Id + MSSubClass + LotFrontage + LotArea + OverallQual + 
    OverallCond + YearBuilt + YearRemodAdd + MasVnrArea + BsmtFinSF1 + 
    BsmtFinSF2 + BsmtUnfSF + TotalBsmtSF + X1stFlrSF + X2ndFlrSF + 
    LowQualFinSF + GrLivArea + BsmtFullBath + BsmtHalfBath + 
    FullBath + HalfBath + BedroomAbvGr + KitchenAbvGr + TotRmsAbvGrd + 
    Fireplaces + GarageYrBlt + GarageCars + GarageArea + WoodDeckSF + 
    OpenPorchSF + EnclosedPorch + X3SsnPorch + ScreenPorch + 
    PoolArea + MiscVal + MoSold + YrSold

Complete :
            (Intercept) Id MSSubClass LotFrontage LotArea OverallQual
TotalBsmtSF 0           0  0          0           0       0          
GrLivArea   0           0  0          0           0       0          
            OverallCond YearBuilt YearRemodAdd MasVnrArea BsmtFinSF1 BsmtFinSF2
TotalBsmtSF 0           0         0            0          1          1         
GrLivArea   0           0         0            0          0          0         
            B

"complete" tells us that TotalBsmtSF is completely linearly dependent on BsmtFinSF1, BsmtFinSF2, and BsmtUnfSF, and similarly GrLivArea on X1stFlrSF, X2ndFlrSF, and LowQualFinSF.

## 3. Interactive effects

I use the : operator to inspect the interation between TotRmsAbvGrd and FullBath. I conjecture that it is desireable to have a more even ratio of bedrooms and bathrooms, and so testing the interaction between bedrooms and bathrooms might tell us more about the effects of those terms together, and relative to each other.

In [39]:
mod1 <- lm(SalePrice ~ TotRmsAbvGrd + FullBath + TotRmsAbvGrd : FullBath , data = df)

summary(mod1)



Call:
lm(formula = SalePrice ~ TotRmsAbvGrd + FullBath + TotRmsAbvGrd:FullBath, 
    data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-206352  -30575   -5034   21221  402874 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)            91027.2    19354.8   4.703 2.81e-06 ***
TotRmsAbvGrd            -385.7     3186.7  -0.121    0.904    
FullBath               -4176.4    11430.9  -0.365    0.715    
TotRmsAbvGrd:FullBath   9249.5     1696.5   5.452 5.83e-08 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 61690 on 1456 degrees of freedom
Multiple R-squared:  0.3983,	Adjusted R-squared:  0.397 
F-statistic: 321.2 on 3 and 1456 DF,  p-value: < 2.2e-16


## 4. Data transformations

We can do quick on the fly transformations inside the lm function.  
  
  
One common reason to use Log(var) in a regression is to capture decreasing returns to scale, succh that as a variable increases, there are diminishing effects on the dependent variable. One variable here that might exhibit that is GaragCars, where going from 3 to 4 car garages might be less of an effect than going from 0 to 1. Since $log(0) = \text{NA}$ we add 1 to offset **(Double transformation!)**  
  
Another common technique is to add $\alpha^{2}$ to a regression to capture a variable whose effect might reverse, such that the effect is modeled by a parabola. We might try adding this term to the YearBuilt variable if we thoughbt that antique (very old) houses might add value, but middle-aged houses are worthless

In [40]:
mod2 <- lm(SalePrice ~ log(1 + GarageCars) + FullBath + OverallQual , data = df)
summary(mod2)

mod2 <- lm(SalePrice ~  YearBuilt^2 + FullBath + OverallQual , data = df)
summary(mod2)



Call:
lm(formula = SalePrice ~ log(1 + GarageCars) + FullBath + OverallQual, 
    data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-182022  -27601   -3276   20586  388228 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)          -108539       5510 -19.700  < 2e-16 ***
log(1 + GarageCars)    41656       4566   9.124  < 2e-16 ***
FullBath               21683       2652   8.175 6.37e-16 ***
OverallQual            35252       1152  30.611  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 45880 on 1456 degrees of freedom
Multiple R-squared:  0.6672,	Adjusted R-squared:  0.6665 
F-statistic: 972.8 on 3 and 1456 DF,  p-value: < 2.2e-16



Call:
lm(formula = SalePrice ~ YearBuilt^2 + FullBath + OverallQual, 
    data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-186037  -27741   -2967   20841  387645 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -434766.09   96580.20  -4.502 7.28e-06 ***
YearBuilt       175.80      50.95   3.450 0.000576 ***
FullBath      23779.61    2744.07   8.666  < 2e-16 ***
OverallQual   38022.85    1177.82  32.282  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 46980 on 1456 degrees of freedom
Multiple R-squared:  0.651,	Adjusted R-squared:  0.6503 
F-statistic: 905.2 on 3 and 1456 DF,  p-value: < 2.2e-16
